In [ ]:
from dotenv import load_dotenv
import openai
import os

load_dotenv()  # Load environment variables from .env

client = openai.OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
# If you didn't set an environment variable, you could do:
# client = openai.OpenAI(api_key="sk-your-api-key")  # (Not recommended to hard-code in real apps)

# Define a function to generate a response from the AI given a user message
def generate_response(user_prompt):
    """
    Sends the user prompt to OpenAI and returns the AI's response.

    Parameters:
    -----------
    user_prompt : str
        The input message from the user.

    Returns:
    --------
    str
        The AI-generated response as plain text.
    """
    # Use OpenAI's chat completion endpoint to get a chat-based response
    response = client.chat.completions.create(
        model="gpt-4o",  # The AI model to use
        messages=[{"role": "user", "content": user_prompt}]  # The conversation context
    )
    # Extract the assistant's message from the response
    message_text = response.choices[0].message.content
    return message_text  # Return the assistant's reply

# Quick test: let's see if the function works by asking a sample question
test_reply = generate_response("Hello, how are you?")
print(test_reply)  # This should print an AI-generated response, e.g., "Hello! I'm doing well, how can I assist you?"


In [ ]:
import streamlit as st  # Import Streamlit for the UI

# Set the configuration of the page
st.set_page_config(
    page_title="AI Chatbot",   # Title of the web page
    page_icon="🤖",           # An icon for the page (emoji of a robot)
    layout="wide"             # Use the full width of the page for a wide layout
)

# Add a title to the app
st.title("🤖 AI Chatbot Assistant")
st.markdown("**Welcome!** Ask anything or upload a file for the bot to analyze.")

In [ ]:
# Sidebar section for file upload
uploaded_file = st.sidebar.file_uploader(
    "Upload a file (optional):",  # Label for the uploader
    type=["txt", "pdf"]           # Limit file types to text or PDF for this example
)

# If a file is uploaded, we can read or process it (here we just show the file name for confirmation)
if uploaded_file is not None:
    file_details = f"**{uploaded_file.name}** ({uploaded_file.size} bytes)"
    st.sidebar.write("Uploaded file:", file_details)
    # In a real app, you might pass this file's content to the AI or do something with it.
    # For example, if it's a text file you could do: text = uploaded_file.read().decode('utf-8')

In [ ]:
# Initialize chat history in session state if not already there
if "messages" not in st.session_state:
    st.session_state.messages = []  # list to hold all messages (dicts with 'role' and 'content')

# (Optional) If we want the bot to start with a greeting message, we can add an initial assistant message:
if not st.session_state.messages:
    st.session_state.messages.append({"role": "assistant", "content": "Hello! I'm here to help. Feel free to ask me anything or upload a file for analysis."})

In [ ]:
# Display all past messages in the chat
for msg in st.session_state.messages:
    if msg["role"] == "assistant":
        # Display assistant messages on the left (default)
        with st.chat_message("assistant"):
            st.markdown(msg["content"])
    else:
        # Display user messages on the right side
        with st.chat_message("user"):
            st.markdown(msg["content"])

In [ ]:
# Chat input widget (appears at the bottom of the page)
user_message = st.chat_input("Type your message here...")

if user_message:
    # 1. Add the user message to chat history
    st.session_state.messages.append({"role": "user", "content": user_message})
    
    # 2. Display the user message immediately (so it appears in the chat above)
    with st.chat_message("user"):
        st.markdown(user_message)
    
    # 3. Generate the AI's response (this might take a moment)
    with st.chat_message("assistant"):
        with st.spinner("Thinking..."):  # Show a spinner while waiting for the AI
            assistant_reply = generate_response(user_message)
            st.markdown(assistant_reply)
    # 4. Add the assistant's reply to the chat history
    st.session_state.messages.append({"role": "assistant", "content": assistant_reply})

In [ ]:
# app.py (full code combining all steps)

import os
import openai
import streamlit as st

openai.api_key = os.getenv("OPENAI_API_KEY")

def generate_response(user_prompt):
    response = openai.ChatCompletion.create(
        model="gpt-4o-turbo",
        messages=[{"role": "user", "content": user_prompt}]
    )
    message_text = response["choices"][0]["message"]["content"]
    return message_text

st.set_page_config(page_title="AI Chatbot", page_icon="🤖", layout="wide")
st.title("🤖 AI Chatbot Assistant")
st.markdown("**Welcome!** Ask anything or upload a file for the bot to analyze.")

uploaded_file = st.sidebar.file_uploader("Upload a file (optional):", type=["txt", "pdf"])
if uploaded_file is not None:
    st.sidebar.write("Uploaded file:", f"**{uploaded_file.name}** ({uploaded_file.size} bytes)")

if "messages" not in st.session_state:
    st.session_state.messages = []
if not st.session_state.messages:
    st.session_state.messages.append({"role": "assistant", "content": "Hello! I'm here to help. Feel free to ask me anything or upload a file."})

# Display existing chat messages
for msg in st.session_state.messages:
    with st.chat_message(msg["role"]):
        st.markdown(msg["content"])

# Chat input widget for new messages
if user_msg := st.chat_input("Type your message here..."):
    # Add user message to history and display it
    st.session_state.messages.append({"role": "user", "content": user_msg})
    with st.chat_message("user"):
        st.markdown(user_msg)
    # Generate assistant response with spinner
    with st.chat_message("assistant"):
        with st.spinner("Thinking..."):
            assistant_msg = generate_response(user_msg)
            st.markdown(assistant_msg)
    # Add assistant response to history
    st.session_state.messages.append({"role": "assistant", "content": assistant_msg})

In [ ]:
# Run this in your terminal, not in this notebook
# streamlit run app.py

In [ ]:
from IPython.display import HTML

HTML("""
<video width="960" height="720" controls>
  <source src="assets/streamlit_chatbot_video.mp4" type="video/mp4">
  Your browser does not support the video tag.
</video>
""")
